In [1]:
import numpy as np
import pandas as pd

from time import process_time
from tqdm.auto import tqdm

import re

In [2]:
import requests
from bs4 import BeautifulSoup 
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
import pymongo
from pymongo import MongoClient

In [61]:
path = 'C:/Users/s0970/python/701網路爬蟲/chromedriver.exe'
driver = webdriver.Chrome(path) #開啟

<ipython-input-61-b2cc4c1ac33d>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path) #開啟


In [62]:
driver.get("https://referendum.2021.nat.gov.tw/pc/zh_TW/IDX/indexFF.html")
driver.find_element_by_name("nodeIcon2").click()

html = BeautifulSoup(driver.page_source, 'html.parser')

number = []
for i in html.select('div[id="domRoot"] div[id]'):
    number.append(i.attrs["id"])

pattern = re.compile(r'(?<=folder)\d+')

num = [int(pattern.findall(i)[0]) for i in number[3:25]]

numbers=[]

for j in range(len(num)-1): # 最後一個嘉義市另外看
    numbers.append([num[j]+i+1 for i in range(num[j+1]-num[j]-1)]) 

numbers.append([391,392])

<ipython-input-62-128daabf361a>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_name("nodeIcon2").click()


In [63]:
driver = webdriver.Chrome(path) 

def vote_data(city,adms):
    
    driver.get("https://referendum.2021.nat.gov.tw/pc/zh_TW/IDX/indexFF.html")

    driver.find_element_by_name("nodeIcon2").click()
    driver.find_element_by_name("nodeIcon{}".format(city)).click()
    driver.find_element_by_id("itemTextLink{}".format(adms)).click()

    html = BeautifulSoup(driver.page_source, 'html.parser')


    pattern = re.compile(r'(?<=\xa0-\xa0)\w+')
    Adm = pattern.findall(html.select('td[valign="bottom"]')[0].text)[0]

    pattern = re.compile(r'(?<=\xa0)\d+')
    vote = int(pattern.findall(html.select('td[align="right"]')[0].text)[0])

    df = [i.text for i in html.select('tr.trT td')]

    df.insert(0,Adm)
    df.append(vote)

    return df

<ipython-input-63-c4f99386fca4>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [64]:
cols=["縣市行政區","同意票數","不同意票數","有效票數","無效票數","投票數","投票權人數","已完成投票所投票率(%)","有效同意票數對投票權人數(%)","已完成投票所"]

In [68]:
client = MongoClient('localhost', 27017)
mydb = client["test"]

In [70]:
k=-1
for i in num:
    k+=1
    for j in numbers[k]:
        if  mydb["vote_final"].find_one({"縣市行政區":vote_data(i,j)[0]}):
            print("'{}'資料已新增".format(vote_data(i,j)[0]))
        else:
            try:
                mydb["vote_final"].insert_one(dict(zip(cols,vote_data(i,j))))
            except:
                n = {}
                for k, v in dict(zip(cols,vote_data(i,j))).items():
                    if isinstance(v, np.int64):
                        v = int(v)
                    n[k]= v

                mydb["vote_final"].insert_one(n)

<ipython-input-63-c4f99386fca4>:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_name("nodeIcon2").click()
<ipython-input-63-c4f99386fca4>:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_name("nodeIcon{}".format(city)).click()
<ipython-input-63-c4f99386fca4>:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id("itemTextLink{}".format(adms)).click()
